<a href="https://colab.research.google.com/github/reshmadua/Grocery-Item-detection/blob/main/Grocery%20item%20detection%20and%20bill%20generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

**Clone repo, install dependencies and check PyTorch and GPU**

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla K80)


**Linking Google Drive in Google Colab notebook**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Copying Dataset from Google Drive to a folder in YOLOv5 directory**

In [3]:
%cp /content/gdrive/My\ Drive/dataset.zip /content/dataset.zip

**Unzipping the Dataset zip file**

In [4]:
!unzip -q ../dataset.zip -d ../

# **Training**
Training a YOLOv5s model on custom dataset with --data custom_data.yaml, starting from pretrained --weights yolov5s.pt

In [ ]:
# !python train.py --img 640 --batch 16 --epochs 30 --data custom_data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-431-gfcb225c torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.

**Copy weights from the YOLOv5 notebook saved at runs/train/exp2/weights/best.pt to Google Drive**


In [6]:
# %cp /content/yolov5/runs/train/exp/weights/best.pt /content/gdrive/My\ Drive

# **Loading model weights**
Once the weights are saved in Google Drive we may want to reload them in a later Google Colab Session hence we copy weights from Google Drive to a folder called "weights" in YOLOv5 directory



In [5]:
!mkdir /content/yolov5/weights

In [6]:
%cp /content/gdrive/My\ Drive/best.pt /content/yolov5/weights/best.pt

In [7]:
!mkdir /content/yolov5/Test_Data

**Extracting necessary frames from video to perform object detection** 

The results are saved to runs/detect

In [8]:
import cv2
import os 

video = cv2.VideoCapture('/content/gdrive/MyDrive/test.mp4')
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('Writing frames to files')
for fno in range(0, total_frames, fps):
    video.set(cv2.CAP_PROP_POS_FRAMES, fno)
    ret, frame = video.read()
    if ret:
        name = '/content/yolov5/Test_Data/Frame(' + str(fno) + ').jpg'
        cv2.imwrite(name, frame)

s = !python detect.py --weights /content/yolov5/weights/best.pt --img 640 --conf 0.25 --source /content/yolov5/Test_Data/
print(s)

print('Clearing the file content')

# Reading all the files in Test_data and deleting it
for file_name in os.listdir('/content/yolov5/Test_Data'):
    if file_name.endswith('.jpg'):
        os.remove('/content/yolov5/Test_Data/' + file_name)        
print('File content successfully cleared')

Writing frames to files
['Downloading https://ultralytics.com/assets/Arial.ttf to /root/.config/Ultralytics/Arial.ttf...', "\x1b\x1bdetect: \x1bweights=['/content/yolov5/weights/best.pt'], source=/content/yolov5/Test_Data/, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False", 'YOLOv5 🚀 v5.0-483-g76d301b torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)', '', 'Fusing layers... ', 'Model Summary: 224 layers, 7064698 parameters, 0 gradients, 16.4 GFLOPs', 'image 1/3 /content/yolov5/Test_Data/Frame(0).jpg: 384x640 1 chocolate, 2 softdrinks, Done. (0.025s)', 'image 2/3 /content/yolov5/Test_Data/Frame(30).jpg: 384x640 2 chipss, 1 maggi, Done. (0.024s)', 'image 3/3 /content/yolov5/Test_Data/Frame(60).j

**Filtering out the outcomes in a dictionary**

In [9]:
# Creating initial filtered list for the outcome
actual_list = []
for i in range(len(s)):
  if('Done' in s[i]):
    actual_list.append(s[i])

# Creating a dictionary for the outcome
bill = {}
classes = ['chocolate,','softdrinks,','chips,','icecream,', 'maggi,']
for i in range(len(actual_list)):
  sub_list = list(actual_list[i].split(' '))
  for j in range(len(sub_list)):
    if(sub_list[j] in classes):

# Updating value for each class in the dictionary
      if(sub_list[j] in list(bill.keys())):
        bill[sub_list[j]] = bill[sub_list[j]] + int(sub_list[j-1])

# Initialising the class values in the dictionary
      else:
        bill[sub_list[j]] = int(sub_list[j-1])


**Generating bill for all the grocery items detected**

In [10]:
import pandas as pd

# per unit cost for each item 
price = {'chocolate,' : 60 ,
         'softdrinks,' : 45,
         'chips,': 10,
         'icecream,': 120,
         'maggi,': 15}

# Creating the bill in a Dataframe
df = pd.DataFrame()
df['item(s)']=bill.keys()
df['quantity']=bill.values()
price_l=[]

# Calculating the price of each item
for i in range(len(df)):
  #print(df['item(s)'][i])
  p=price[df['item(s)'][i]]
  q=df['quantity'][i]
  p=p*q
  price_l.append(p)

# Adding the price values to the dataframe 
df['price']=price_l

# Displaying the output in a formatted way
print("Bill Generated"+'\n',df)
print("\n\n"+"Grand total: ",sum(df['price']))

Bill Generated
        item(s)  quantity  price
0   chocolate,         1     60
1  softdrinks,         2     90
2       maggi,         1     15
3       chips,         1     10
4    icecream,         1    120


Grand total:  295
